In [ ]:
# Import libraires 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Breif summary about this in Notebook.Below are few topcis we will cover in this notebook.
* Step by Step EDA
* Handel imbalanced dataset
* Model Selection
* Hyperparameter tuning
    


In [ ]:
data = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv") #Load the dataset

Let's take a look at basic statisics of the data like mean,min,max,standard deviation,percentile of the data

In [ ]:
data.describe()

From the above table we can see that min values for most of the columns are 0. That's highlhy unlikely. Let's look at some of the top and bottom rows in the data

In [ ]:
data.head(10)

In [ ]:
data.tail(10) 

When we look at the data.describe method we can see Insulin,Gluscose,Blood Pressure,SkinThickness and BMI are 0 
But they can't be zero. They zero values could be be becuase of missing values/data corruption
Beofer we jump to any conclusion let see the count of missing values in the datset

In [ ]:
data.isnull().sum()
#There are no missing values in the dataset but there are 0 values.

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data.corr(),annot=True,linewidths=3)

plt.show()

Let's replace the 0 values in datasent  with numpy Nan.
In later part of the notebook will see how to repalce the incorrect/missing values

In [ ]:
for col in data.columns:
    if col !="Pregnancies" and col !="DiabetesPedigreeFunction" and col !="Outcome":
        data[col] = data[col].replace(0,np.nan)

#We have replaced the inaccure values with nan values.Now let's look for the missing values        
total_missing_values = pd.DataFrame(data.isnull().sum()) # count the percentage of misisng values

percentage_mising_values = pd.DataFrame(data.isnull().sum() / data.shape[0] * 100) #Compute percentage of missing values

missing_df = pd.DataFrame(index=data.columns,columns = ['Total missing values','Total missing values %'])
missing_df['Total missing values'] = total_missing_values
missing_df['Total missing values %'] = percentage_mising_values 
missing_df

As a rule of thumb if any feature have more than 30 % of missing data we can drop it becuase imputing the missing values for these columns will not add much value while using mean,median or mode methods. Having said that we can use some advnaced technquies like KNN or Random forest regressor to predict the missing values (Here the column with missing values will become our dependant variable). But in our cases the correlation between SkinThickness,Insulin is very less so we can drop them.

In [ ]:
data.drop(['Insulin','SkinThickness'],inplace=True,axis=1)

Before imputing the missing values let's the distribtion of the cloumns

In [ ]:
data.hist(figsize=(10,10),grid=[3,3])
plt.show()

BMI,Glucose and BloodPressure columns have missing values. From above plot we can see that these features follow so what a normal distribution. We can impute the missing values with the mean of the columns

In [ ]:
data['BMI'].fillna(data['BMI'].mean(),inplace=True)
data['Glucose'].fillna(data['Glucose'].mean(),inplace=True)
data['BloodPressure'].fillna(data['BloodPressure'].mean(),inplace=True)

## Univariate analysis

Let's see the distribution of the feautes once again after imputation

In [ ]:
data.hist(figsize=(10,10),grid=[4,4])
plt.show()

Age,DiabetesPedigreeFunction and Pregnanices featues seems to be swked. Let anaylse these feautes

In [ ]:
sns.distplot(data['Age'])
plt.show()

In [ ]:
sns.boxplot(x='Outcome',y='Age',data=data)
plt.show()

In [ ]:
data[data['Age'] > 68]

In [ ]:
data.drop(data[data['Age'] > 68].Age,inplace=True)

In [ ]:
sns.boxplot(y='DiabetesPedigreeFunction',x='Outcome',data=data)
plt.show()

In [ ]:
sns.boxplot(y='Pregnancies',x='Outcome',data=data)
plt.show()

Having more than 10 pregnancies is higlhy unlikely.This could be beacuse of typo error.Let's remove them

In [ ]:
data.drop(data[data['Pregnancies'] >10].index,inplace=True)

Let's take a closer look at the dependant variable

In [ ]:
data['Outcome'].value_counts()

In [ ]:
sns.distplot(data['Outcome'])
plt.show()


This dataset is imblanced . That means there are more rows with 0 label than 1.This will bias the prediction model towards the more common class.
Take a look at https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18 to under more on imbalanced 
dataset and how it will effect the model performance
In this note book we will measure the model performance on Orginal, downsampled and syntheticaly generated sampled datasets

In [ ]:
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

imbalanced = data.copy(deep=True)
downsampled = data.copy(deep=True)
synthetic_dataset = data.copy(deep=True)

#Before we down sample the data we should split the data based on class labels.
not_diabetic = downsampled[downsampled.Outcome==0]
diabetic = downsampled[downsampled.Outcome==1]

not_diabetic_downsampled = resample(not_diabetic,
                                replace = False, # sample without replacement
                                n_samples = len(diabetic), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled  = pd.concat([not_diabetic_downsampled, diabetic])

#split the data into x and y
synthetic_dataset_X = synthetic_dataset.iloc[:,:-1]
synthetic_dataset_y = synthetic_dataset.iloc[:,-1]
sm = SMOTE(random_state = 10) 
#Fit the SMOTE model to the data
synthetic_X, synthetic_y = sm.fit_sample(synthetic_dataset_X, synthetic_dataset_y.ravel()) 
df_synthetic_dataset_y = pd.DataFrame(columns=['Outcome'])
df_synthetic_dataset_y['Outcome'] = synthetic_y
synthetic_dataset_X = pd.DataFrame(synthetic_X, columns=synthetic_dataset.columns[:-1])
#Concat X and Y again into singel dataset
synthetic_dataset = pd.concat([synthetic_dataset_X,df_synthetic_dataset_y],axis=1) 

print("Class labels of imbalanced dataset has {} 0s and {} 1s.\n".format(imbalanced['Outcome'].value_counts()[0],imbalanced['Outcome'].value_counts()[1]))
print("Class labels of downsampled dataset has {} 0s and {} 1s.\n".format(downsampled['Outcome'].value_counts()[0],downsampled['Outcome'].value_counts()[1]))
print("Class labels of synthetic dataset has {} 0s and {} 1s.\n".format(synthetic_dataset['Outcome'].value_counts()[0],synthetic_dataset['Outcome'].value_counts()[1]))

## Bivariate analysis

In [ ]:
plt.figure(figsize=(10,5))
sns.pairplot(data=data,hue='Outcome')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data.corr(),annot=True,linewidths=3)

plt.show()

From above heat map we can see Gloucose and BMI are one the feautre which is highly corelated with Outcome

All the feautres in the data are in different scale. So we have to scale the values.

## Modle Selection

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report

datasets = [imbalanced,downsampled,synthetic_dataset]
dataset_names = ['imbalanced','downsampled','synthetic_dataset']
models = [KNeighborsClassifier,LogisticRegression,RandomForestClassifier]

columns = ['Dataset','Model','accuray_score','f1_score','TN','FP','FN','TP']
Model_details = pd.DataFrame(columns=columns)



for dataset_name in dataset_names:
    index_dataset = dataset_names.index(dataset_name)
    frame = {}
    for model in models:
        frame['Dataset'] = dataset_name
        frame['Model'] = model.__name__
        dataset = datasets[index_dataset]
        X= dataset.iloc[:,:-1]
        y = dataset.iloc[:,-1]
        
        #scale the values
        sc_X = StandardScaler()
        X = sc_X.fit_transform(X)
        #split the dataset into train and test
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42, stratify=y)
        clf = model()
        clf.fit(X_train,y_train)
        
        y_pred = clf.predict(X_test)
        
        frame['f1_score'] = f1_score(y_test,y_pred)
        frame['accuray_score'] = accuracy_score(y_test,y_pred)
        frame['TN'] = confusion_matrix(y_test,y_pred)[0][0]
        frame['FP'] = confusion_matrix(y_test,y_pred)[0][1]
        frame['FN'] = confusion_matrix(y_test,y_pred)[1][0]
        frame['TP'] = confusion_matrix(y_test,y_pred)[1][1]
        Model_details = Model_details.append(frame,ignore_index=True)

In [ ]:
Model_details

### Hyper parameter tuning

From above result clearly KNeighborsClassifier is the better model.
Observe that though accuracy score is high for imbalanced dataset but the f1_score is low.
Synthetic_dataset and downsampled dataset achived high accuray score as well as f1 score.

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {"n_neighbors": np.arange(1, 25),'weights':['uniform','distance']}


X= downsampled.iloc[:,:-1]
y = downsampled.iloc[:,-1]
        
#scale the values
sc_X = StandardScaler()
X = sc_X.fit_transform(X)
#split the dataset into train and test
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42, stratify=y)

clf = KNeighborsClassifier()
clf_cv= GridSearchCV(clf,param_grid,cv=3,n_jobs=-1)
clf_cv.fit(X,y)

In [ ]:
clf_cv.best_params_

We can see that 12 is the optimal value for ‘n_neighbors’ and distance is the weight metric. We can use the ‘best_score_’ function to check the accuracy of our model when ‘n_neighbors’ is 12. ‘best_score_’ outputs the mean accuracy of the scores obtained through cross-validation.

In [ ]:
clf_cv.best_score_